In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from src.infra import config

path = '../config/test.yaml'
opt = config.load_config(path)
opt.network.feature_extractor.args.cache_dir = '../data/FAUST_r/diffusion'
opt.path = path
print(opt)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
device_select: auto
network:
  feature_extractor:
    args:
      cache_dir: ../data/FAUST_r/diffusion
      in_channels: 128
      input_type: wks
      out_channels: 256
    name: DiffusionNet
  fm_solver:
    args:
      
    name: RegularizedFMNet
  name: URSSM
  permutation:
    args:
      tau: 0.07
    name: Similarity
path: ../config/test.yaml
train:
  checkpoint_interval: 1000
  dataset:
    test:
      args:
        data_root: data/FAUST_r/
        num_evecs: 200
        phase: test
        return_corr: True
        return_dist: True
        return_evecs: True
        return_faces: True
      dataloader:
        args:
          batch_size: 1
          shuffle: True
        name: DataLoader
      name: PairFaustDataset
    train:
      args:
        data_root: data/FAUST_r/
        num_evecs: 200
        phase:

In [4]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Dataset

In [5]:
import src
from src.infra.registry import DATASET_REGISTRY
DATASET_REGISTRY

{'Dataset': torch.utils.data.dataset.Dataset,
 'SingleFaustDataset': src.dataset.shape_cor.SingleFaustDataset,
 'SingleScapeDataset': src.dataset.shape_cor.SingleScapeDataset,
 'SingleShrec19Dataset': src.dataset.shape_cor.SingleShrec19Dataset,
 'SingleSmalDataset': src.dataset.shape_cor.SingleSmalDataset,
 'SingleDT4DDataset': src.dataset.shape_cor.SingleDT4DDataset,
 'SingleShrec20Dataset': src.dataset.shape_cor.SingleShrec20Dataset,
 'SingleTopKidsDataset': src.dataset.shape_cor.SingleTopKidsDataset,
 'PairDataset': src.dataset.shape_cor.PairDataset,
 'PairFaustDataset': src.dataset.shape_cor.PairFaustDataset,
 'PairScapeDataset': src.dataset.shape_cor.PairScapeDataset,
 'PairShrec19Dataset': src.dataset.shape_cor.PairShrec19Dataset,
 'PairSmalDataset': src.dataset.shape_cor.PairSmalDataset,
 'PairDT4DDataset': src.dataset.shape_cor.PairDT4DDataset,
 'PairShrec20Dataset': src.dataset.shape_cor.PairShrec20Dataset,
 'PairShrec16Dataset': src.dataset.shape_cor.PairShrec16Dataset,
 'Pai

In [6]:
dataset = DATASET_REGISTRY['PairFaustDataset'](
    phase='train',
    data_root='../data/FAUST_r/',
    return_evecs='true',
    return_faces='true',
    num_evecs=200,
    return_corr='false',
    return_dist='false',
)

In [7]:
from src.infra.registry import DATALOADER_REGISTRY
DATALOADER_REGISTRY

{'DataLoader': torch.utils.data.dataloader.DataLoader}

In [8]:
dataloader = DATALOADER_REGISTRY['DataLoader'](
    dataset=dataset,
    batch_size=1,
    shuffle=True,
)
dataloader

In [9]:
from src.utils.tensor import to_device

for batch, data in enumerate(dataloader):
    data = to_device(data, device)
    data_x, data_y = data['first'], data['second']
    print(data_x['name'], data_y['name'])
    break

['tr_reg_043'] ['tr_reg_013']


In [10]:
data

{'first': {'name': ['tr_reg_043'],
  'verts': tensor([[[ 0.0365,  0.4162,  0.0117],
           [ 0.0524,  0.3562,  0.0013],
           [ 0.0177,  0.3737,  0.0178],
           ...,
           [ 0.1372,  0.1737, -0.0159],
           [ 0.1341,  0.1765, -0.0186],
           [-0.0078, -0.1838, -0.0136]]], device='cuda:0'),
  'faces': tensor([[[  11,    0,   23],
           [  61,    0,   95],
           [  60,    0,   61],
           ...,
           [4993, 2985, 4029],
           [4993, 4029, 4992],
           [2985, 2983, 4029]]], device='cuda:0'),
  'evecs': tensor([[[ 1.0000e+00, -1.1361e+00,  2.0527e-03,  ..., -2.9906e+00,
            -3.1261e+00, -5.4041e-01],
           [ 1.0000e+00, -1.1023e+00,  1.2053e-03,  ..., -9.5013e-01,
            -7.1931e-01, -1.7982e+00],
           [ 1.0000e+00, -1.1157e+00,  2.1572e-03,  ..., -9.9875e-01,
            -2.5395e+00,  3.3135e-01],
           ...,
           [ 1.0000e+00, -9.1606e-01, -2.7488e-02,  ...,  5.6287e-01,
             2.6026e-01, -5

## Model

In [11]:
from src.infra.registry import NETWORK_REGISTRY
NETWORK_REGISTRY

{'URSSM': src.network.urssm.URSSM}

In [12]:
urssm = NETWORK_REGISTRY[opt.network.name](opt).to(device)
urssm

URSSM(
  (feature_extractor): DiffusionNet(
    (first_linear): Linear(in_features=128, out_features=128, bias=True)
    (last_linear): Linear(in_features=128, out_features=256, bias=True)
    (blocks): ModuleList(
      (0): DiffusionNetBlock(
        (diffusion): LearnedTimeDiffusion()
        (gradient_features): SpatialGradientFeatures(
          (A_re): Linear(in_features=128, out_features=128, bias=False)
          (A_im): Linear(in_features=128, out_features=128, bias=False)
        )
        (mlp): MiniMLP(
          (miniMLP_linear_000): Linear(in_features=384, out_features=128, bias=True)
          (miniMLP_activation_000): ReLU()
          (miniMLP_dropout_001): Dropout(p=0.5, inplace=False)
          (miniMLP_linear_001): Linear(in_features=128, out_features=128, bias=True)
          (miniMLP_activation_001): ReLU()
          (miniMLP_dropout_002): Dropout(p=0.5, inplace=False)
          (miniMLP_linear_002): Linear(in_features=128, out_features=128, bias=True)
        )
  

In [13]:
network_path = '../checkpoints/faust.pth'
urssm.feature_extractor.load_state_dict(
    torch.load(network_path)['networks']['feature_extractor']
)
print(f'Loaded pretrain weights from {network_path}')

Loaded pretrain weights from ../checkpoints/faust.pth


In [14]:
infer = urssm.feed(data)
infer

{'Cxy': tensor([[[ 9.7130e-01, -1.5122e-02, -1.8399e-03,  ...,  2.5405e-04,
           -3.2051e-06,  3.4399e-04],
          [ 8.2068e-03, -9.5574e-01,  2.5925e-02,  ...,  3.6873e-04,
           -1.5091e-04,  5.1748e-04],
          [ 8.9977e-03, -2.8313e-02, -9.1457e-01,  ...,  3.3003e-04,
           -1.3187e-04, -6.1882e-04],
          ...,
          [-8.2736e-04, -5.1638e-04, -3.6542e-04,  ..., -4.5998e-02,
            5.6245e-02, -2.6612e-01],
          [ 3.1957e-04,  4.6047e-04, -2.1055e-04,  ...,  1.4296e-01,
           -3.7584e-02,  2.8080e-01],
          [-1.0973e-03,  3.5642e-04, -4.0581e-05,  ...,  1.9760e-01,
           -9.6500e-02,  1.3485e-01]]], device='cuda:0', grad_fn=<CatBackward0>),
 'Cyx': tensor([[[ 9.8565e-01, -1.3447e-02, -1.2126e-03,  ..., -1.1071e-04,
            8.1987e-04, -1.0983e-04],
          [ 9.1590e-03, -1.0021e+00, -3.5750e-02,  ..., -1.5244e-03,
           -5.2046e-04,  6.3162e-04],
          [ 9.3044e-03,  2.8630e-02, -1.0561e+00,  ..., -1.9739e-04,
  

## Loss

In [15]:
from src.infra.registry import LOSS_REGISTRY
LOSS_REGISTRY

{'BijectivityLoss': src.loss.fmap.BijectivityLoss,
 'OrthogonalityLoss': src.loss.fmap.OrthogonalityLoss,
 'LaplacianCommutativityLoss': src.loss.fmap.LaplacianCommutativityLoss,
 'SpatialSpectralAlignmentLoss': src.loss.fmap.SpatialSpectralAlignmentLoss,
 'PartialFmapsLoss': src.loss.fmap.PartialFmapsLoss,
 'DirichletLoss': src.loss.dirichlet.DirichletLoss}

In [16]:
loss_dict = {}

for name, loss in opt.train.loss.items():
    loss_dict[name] = {
        'fn': LOSS_REGISTRY[loss['name']](**loss['args']).to(device),
        'weight': loss['weight'],
    }

loss_dict

{'bij_loss': {'fn': BijectivityLoss(
    (squared_frobenius): SquaredFrobeniusLoss()
  ),
  'weight': 1.0},
 'orth_loss': {'fn': OrthogonalityLoss(
    (squared_frobenius): SquaredFrobeniusLoss()
  ),
  'weight': 1.0},
 'align_loss': {'fn': SpatialSpectralAlignmentLoss(
    (squared_frobenius): SquaredFrobeniusLoss()
  ),
  'weight': 1.0}}

In [17]:
for name, loss in loss_dict.items():
    print(name, loss['fn'].feed(infer, data))

bij_loss tensor(17.4373, device='cuda:0', grad_fn=<AddBackward0>)
orth_loss tensor(48.4561, device='cuda:0', grad_fn=<AddBackward0>)
align_loss tensor(53.4464, device='cuda:0', grad_fn=<AddBackward0>)


## Metrics

In [18]:
from src.infra.registry import METRIC_REGISTRY
METRIC_REGISTRY

{'MeanDiffRatio': src.metric.stats.MeanDiffRatio,
 'StdDiffRatio': src.metric.stats.StdDiffRatio,
 'GeodesicDist': src.metric.geodist.GeodesicDist}

In [19]:
metric_dict = {}

for name, metric in opt.train.metric.items():
    metric_dict[name] = METRIC_REGISTRY[metric['name']](**metric['args']).to(device)

metric_dict

{'geodist': GeodesicDist()}

In [20]:
for name, metric in metric_dict.items():
    metric.eval()
    metric.start_feed(None, 'geodist')
    metric.feed(infer, data)

## Training loop

In [21]:
from src.infra.registry import OPTIMIZER_REGISTRY
OPTIMIZER_REGISTRY

{'SGD': torch.optim.sgd.SGD,
 'Adam': torch.optim.adam.Adam,
 'AdamW': torch.optim.adamw.AdamW,
 'RMSprop': torch.optim.rmsprop.RMSprop,
 'Adagrad': torch.optim.adagrad.Adagrad,
 'Adadelta': torch.optim.adadelta.Adadelta}

In [22]:
optimizer = OPTIMIZER_REGISTRY[opt.train.optimizer.name](
    params=urssm.parameters(),
    **opt.train.optimizer.args,
)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [23]:
urssm.train()

for batch, data in enumerate(dataloader):
    data = to_device(data, device)
    infer = urssm(data)
    loss_val = 0
    
    for name, loss in loss_dict.items():
        loss_value = loss['fn'](infer, data) * loss['weight']
        loss_val += loss_value
    
    print(f'batch {batch} loss: {loss_val.item()}')

    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

    if batch > 10:
        break

batch 0 loss: 156.3905029296875
batch 1 loss: 161.17811584472656
batch 2 loss: 131.73617553710938
batch 3 loss: 188.54330444335938
batch 4 loss: 147.65383911132812
batch 5 loss: 200.42495727539062
batch 6 loss: 154.72117614746094
batch 7 loss: 154.2474822998047
batch 8 loss: 135.6914520263672
batch 9 loss: 149.32748413085938
batch 10 loss: 158.50827026367188
batch 11 loss: 180.33425903320312


## Inference mode

In [24]:
infer = urssm.inference(data)
infer

{'Cxy': tensor([[[-9.6680e-01, -7.6220e-03, -5.4017e-03,  ..., -3.8725e-04,
            5.4300e-04, -1.3607e-04],
          [-8.4055e-03,  9.5202e-01, -9.6099e-02,  ..., -3.6205e-04,
            2.4001e-05, -6.0918e-04],
          [ 4.7266e-03,  1.5281e-01,  9.4750e-01,  ..., -9.4869e-04,
           -2.7597e-04, -2.7404e-04],
          ...,
          [-1.7067e-03, -5.2618e-05,  1.9227e-04,  ..., -8.9134e-02,
            1.8322e-01,  1.3075e-01],
          [-9.6023e-04, -8.8564e-04,  3.0949e-03,  ...,  3.2237e-03,
            3.0350e-01,  2.2264e-01],
          [-2.4657e-04,  1.4766e-03, -3.0290e-04,  ..., -2.8351e-02,
           -1.3229e-01,  7.4403e-02]]], device='cuda:0', grad_fn=<CatBackward0>),
 'Pyx': [tensor([[ 2.0774e-02,  3.1640e-02,  2.9294e-03,  ..., -4.9916e-05,
            4.7147e-05,  4.9359e-05],
          [ 2.2859e-02,  3.2898e-02,  5.4672e-03,  ..., -5.1118e-05,
            1.1208e-05,  2.3583e-05],
          [ 2.3053e-02,  3.3987e-02,  6.0531e-03,  ..., -5.9679e-05,
  